## installation 
* annaconda -d
* jupyter -d
* numpy
* panda
* tensorflow
* opencv
* unrar
* conda
* pycharm

* Keras>=2.0.2
* numpy>=1.12.1
* pandas>=0.19.2
* tqdm>=4.11.2
* matplotlib>=2.0.0
* Pillow>=2.1.0
* h5py>=2.7.0
* ffmpeg
* matplotlib.pyplot as plt
* import mpl_toolkits.mplot3d.axes3d as p3

MSRDailyActivity3D (MSR Daily Activity 3D dataset): 

The dataset was captured by using a Kinect device. There are 16 activities: 
1. drink, 
2. eat, 
3. read book, 
4. call cellphone, 
5. write on a paper, 
6. use laptop, 
7. use vacuum cleaner, 
8. cheer up, 
9. sit still,
10. toss paper,
11. play game, 
12. lie down on sofa,
13. walk, 
14. play guitar,
15. stand up, 
16. sit down. 

There are 10 subjects. Each subject performs each activity twice,  
once in standing position, and once in sitting position. 

There is a sofa in the scene. Three channels are recorded: depth maps (.bin), skeleton joint positions (.txt), and RGB video (.avi). 

There are 16*10*2=320 files for each channel.  

In total, there are 320*3=960 files. Note that the RGB channel and depth channel are recorded independently, so they are not strictly synchronized.

The format of the skeleton file is as follows. 
* The first integer is the number of frames. 
* The second integer is the number of joints which is always 20. 
* For each frame, the first integer is the number of rows. 
* This integer is 40 when there is exactly one skeleton being detected in this frame. 
* It is zero when no skeleton is detected. 
* It is 80 when two skeletons are detected (in that case which is rare, we simply use the first skeleton in our experiments). 
* For most of the frames, the number of rows is 40. Each joint corresponds to two rows. 
* The first row is its real world coordinates (x,y,z) and the second row is its screen coordinates plus depth (u, v, depth) where u and v are normalized to be within [0,1]. 
* For each row, the integer at the end is supposed to be the confidence value, but it is not useful.

## Data Read

In [9]:
# change the working directory
# access to depth, rgb and skeleton files 
rgb_data = 'data/MSRDailypack1/rgb'
rgb_frames = 'data/MSRDailypack1/rgb_frames'
skeleton_data = 'data/MSRDailypack1/skeleton'
depth_data = 'data/MSRDailypack1/depth'

## Extracting frames from videos

In [41]:
test = []
train = []

dataset = glob.glob(rgb_data+'/*')
train = glob.glob(rgb_data+'/train/*')
test = glob.glob(rgb_data+'/test/*')

In [61]:
def get_training_data(path):# path to data folder
    train = glob.glob(path+'/train/*')
    test = glob.glob(path+'/test/*')
    
    with open('listfile.txt', 'w') as filehandle:
        for listitem in train:
            filehandle.write('%s\n' % listitem)    
    
    return train, test    

In [82]:
train = glob.glob(rgb_data+'/train/*')
test = glob.glob(rgb_data+'/test/*')    
with open('listfile.txt', 'w') as filehandle:
    for listitem in train:
        instance = listitem.split('/')[4]
        #print(instance.split('_')[0].split('a')[1])
        filehandle.write('%s %s \n' % listitem,instance.split('_')[0].split('a')[1])  

TypeError: not enough arguments for format string

In [66]:
pwd

'/home/mbs/workspace'

In [56]:
train, test = get_training_data(rgb_data)

In [46]:
import numpy as np
import cv2
import glob
import csv
import os
import os.path

# Access to data folder
#ls data/MSRDailypack1/

#folder = glob.glob(rgb_data+'/*')
folder = train
nb_vid = len(folder)

seqs = []

for vid in folder:
    video_path = vid

    cap = cv2.VideoCapture(video_path)
    print (video_path)

    vid_folder = video_path.split("/")[3].split('.')[0]

    seq = []

    ret, frame = cap.read()

    ret = True        
    count = 0

    create_folder = rgb_frames + "/"+vid_folder
    os.mkdir(create_folder)

    while ret:    
        cap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))  # 1000 (1sec) 500 (0.5sec)
        #print(frame)
        #print ('Read a new frame: ', ret)
        seq.append(frame)    # add this frame to a make sequence
        #print(rgb_frames + "/frames/"+vid_folder+"/frame%d.jpg" % count)    

        cv2.imwrite( rgb_frames + "/"+vid_folder+"/frame%d.jpg" % count, frame)     # save frame as JPEG file    
        #cv2.imwrite(dest,frame)
        count = count + 1
        ret,frame = cap.read()
    
    print(seq)
    seqs.append(seq) # add this sequence of frames to sequence array

print("Out")
print(len(seqs))
# [train|test], class, filename, nb frames

data/MSRDailypack1/rgb/train/a05_s06_e02_rgb.avi
[array([[[  5,  26,  25],
        [  5,  26,  25],
        [  5,  26,  25],
        ...,
        [136, 124, 136],
        [136, 124, 136],
        [136, 124, 136]],

       [[  5,  26,  25],
        [  5,  26,  25],
        [  5,  26,  25],
        ...,
        [136, 124, 136],
        [136, 124, 136],
        [136, 124, 136]],

       [[ 13,  24,  25],
        [ 13,  24,  25],
        [ 13,  24,  25],
        ...,
        [136, 124, 136],
        [136, 124, 136],
        [136, 124, 136]],

       ...,

       [[  0,   1,   0],
        [  0,   1,   0],
        [  0,   1,   0],
        ...,
        [ 84,  85,  95],
        [ 84,  85,  95],
        [ 84,  85,  95]],

       [[  0,   1,   0],
        [  0,   1,   0],
        [  0,   1,   0],
        ...,
        [ 84,  85,  95],
        [ 84,  85,  95],
        [ 84,  85,  95]],

       [[  0,   1,   0],
        [  0,   1,   0],
        [  0,   1,   0],
        ...,
        [ 84,  85,  95],

FileExistsError: [Errno 17] File exists: 'data/MSRDailypack1/rgb_frames/train'

## Saving sequences to npy and then loading them back

In [13]:
# conver list to numpy and save it Numpy format
import numpy
from numpy import save
  
# initilizing list 
#seq
  
# converting list to array 
seqs_np = numpy.array(seqs) 
  
# displaying list 
#print ("List: ", seqs) 
  
# displaying array 
#print ("Array: ", seqs_np) 
print(len(seqs_np))
save('seqs.npy', seqs_np)

320


In [14]:
# load numpy format
from numpy import load


data = np.load('seqs.npy',allow_pickle=True)
# print the array
print(len(data))

320


In [30]:
l = data[0]

for i in l: 
    for r in l[i]: #640
        for c in l[i][c]: #480
            print(len(l[i][r][c]))

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
vid_folder = video_path.split("/")[3].split('.')[0]
count = 1
vid_folder
print(rgb_data + "/frames/"+vid_folder+"/frame%d.jpg" % count)
rgb_frames = 'data/MSRDailypack1/rgb_frames'
print(rgb_frames + vid_folder+"/frame%d.jpg" % count)    

In [ ]:
rgb_frames
frames_dir = glob.glob(rgb_frames+'/*')[0]
frames = glob.glob(frames_dir+'/*')
frames

In [ ]:
for s in seqs:
    for f in s:
        print(f)

In [ ]:
del data # delete numpy array

## Plotting skeleton data

In [ ]:
if __name__=="__main__":
    a = argparse.ArgumentParser()
    a.add_argument("--pathIn", help="path to video")
    a.add_argument("--pathOut", help="path to images")
    args = a.parse_args()
    print(args)
    extractImages(args.pathIn, args.pathOut)

In [1]:
from os import listdir
from os.path import join
import numpy as np
import bunch

In [2]:
def full_fname2_str(data_dir, fname, sep_char):
    fnametostr = ''.join(fname).replace(data_dir, '')
    ind = int(fnametostr.index(sep_char))
    label = int(fnametostr[ind + 1:ind + 3])
    return label

def frame_normalizer(frame, frame_size):
    assert frame.shape[0] == frame_size
    frame = frame.reshape(frame_size // 3, 3)
    spine_mid = frame[1]
    new_frame = []
    j = 0
    for joint in frame:
        new_frame.append(joint - spine_mid)
        j += 1
    new_frame = np.asarray(new_frame)
    return (list(new_frame.flatten()))


def read(data_dir):
    print('Loading MSR 3D Data, data directory %s' % data_dir)
    data, labels, lens, subjects = [], [], [], []
    filenames = []
    documents = [join(data_dir, d)
                 for d in sorted(listdir(data_dir))]
    filenames.extend(documents)
    filenames = np.array(filenames)

    for file in filenames:
        #print("file",file)
        action = np.loadtxt(file)[:, :3].flatten()
        #print("action",action)

        labels.append(full_fname2_str(data_dir, file, 'a'))
        frame_size = len(action) // 60  # 20 iskeleton num x,y,z 3D points
        lens.append(frame_size)
        action = np.asarray(action).reshape(frame_size, 60)
        #print("action shape",action.shape)
        
        new_act = []
        for frame in action:
            new_act.append(frame)

        data.append(new_act)
        subjects.append(full_fname2_str(data_dir, file, 's'))
        
    data = np.asarray(data)
    labels = np.asarray(labels) -1
    lens = np.asarray(lens)
    
    subjects = np.asarray(subjects)
    print("All files read!")
    print('initial shapes [data label len]: %s %s %s' % (data.shape, labels.shape, lens.shape))
    return data,labels,lens

In [3]:
MSR_data_dir = 'dataset/MSRAction3D/MSRAction3DSkeleton(20joints)/'

dataset = read(MSR_data_dir)

Loading MSR 3D Data, data directory dataset/MSRAction3D/MSRAction3DSkeleton(20joints)/


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/MSRAction3D/MSRAction3DSkeleton(20joints)/'

In [ ]:
data,labels,lens = dataset

In [ ]:
np.asarray(data[0]).shape

In [ ]:
action = 16 #jogging

subjects_num = 10 #9
move_num = 3

action_total = action*subjects_num*move_num
print(action_total)

start_a = (action-1)*subjects_num*move_num
end_a = start_a + (subjects_num*move_num)

print("start_a",start_a)
print("end_a",end_a)

action_data = data[start_a : end_a]

In [ ]:
action_data.shape

In [ ]:

total_frames_in_action = 0

print("(frame,20joints*3)")
for a in action_data:
    selected_data = np.asarray(a)
    print(selected_data.shape)
    
    total_frames_in_action+=selected_data.shape[0]
    
print("total_frames_in_action",total_frames_in_action)

# Plotting

In [ ]:
%matplotlib notebook
import pandas
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import sys

In [ ]:
def loadData(action, subject, instance):
    ins = np.loadtxt(basePath + 'a%02i_s%02i_e%02i_skeleton.txt'%(action, subject, instance))
    print("ins.shape[0]",ins.shape[0])
    ins = ins.reshape((ins.shape[0]//20, 20, 4))
    return ins

def animate(i):
    graph._offsets3d = (data[i,:,0], data[i,:,2], data[i,:,1])
    return graph

In [ ]:
basePath = 'dataset/MSRAction3D/MSRAction3DSkeleton(20joints)/'

action = 14 #tennis serve
subject = 1
instance = 1

data = loadData(action, subject, instance)
print(data.shape)

In [ ]:
fig = plt.figure()
ax = p3.Axes3D(fig)

# Setting the axes properties
ax.set_xlim3d([0.0, 300.0])
ax.set_xlabel('X')

ax.set_ylim3d([0.0, 1400.0])
ax.set_ylabel('Z')

ax.set_zlim3d([300.0, 0.0])
ax.set_zlabel('Y')

graph = ax.scatter([], [], [])


anim = FuncAnimation(fig, animate, frames=data.shape[0], interval=100)
plt.show()